
## 1. 为什么BERT在第一句前会加一个[CLS]标志?

引自论文：<BERT: Pre-training of Deep Bidirectional Transformers for
Language Understanding>

The first
token of every sequence is always a special classification token ([CLS]). The final hidden state
corresponding to this token is used as the aggregate sequence representation for classification
tasks.

BERT在第一句前加一个[CLS]标志，最后一层该位对应向量可以作为整句话的语义表示，从而用于下游的<b>分类任务</b>。
[CLS]与文本中已有的其它词相比，这个无明显语义信息的符号会更"公平"地融合文本中各个词的语义信息，从而更好的表示整句话的语义。


## 2. BERT的训练任务有几个，具体描述下每个任务的内容？


BERT训练任务由两个自监督任务组成

+ 1.Masked Language Model:
    + we simply mask some percentage of the input tokens at random, and then predict those masked
tokens; mask 15% of all WordPiece tokens in each sequence at random; only predict the masked words rather than reconstructing the entire input
    + 一个序列在指定MASK数量之后，有80%被真正MASK，10%还是保留原来token，10%被随机替换成其他token，随机词替换会给模型增加一点点噪声，模型不知道哪个词是被随机换了(不然全部用mask模型可能会去学习mask的位置信息)，迫使模型去更好地保留每个词的词义。这个过程类似于完形填空题，可以更好的考虑上下文。
    + 设置一些干扰去预测这个词：例如输入序列The dog is cute.
        + 80% mask掉，需要预测为dog
        + 10% 输入其他词如cat 需要预测为dog
        + 10% 输入dog 需要预测为dog
+ 2.Next Sentence Prediction (NSP):
    + In order to train a model that understands sentence relationships, we pre-train for a binarized next sentence prediction task that can be trivially generated from any monolingual corpus. 
    + 判断句子B是否是句子A的下文。如果是的话输出’IsNext‘，否则输出’NotNext‘。训练数据的生成方式是从平行语料中随机抽取的连续两句话，其中50%保留抽取的两句话，它们符合IsNext关系，另外50%的第二句话是随机从预料中提取的，它们的关系是NotNext的。
    + 在最终隐藏层的计算结果里, 只要取出[cls]token所对应的一条向量, 里面就含有整个句子的信息, 因为期望这个句子里面所有信息都会往[cls]token所对应的一条向量里汇总。
    
最终的Loss = Loss1 + Loss2 作为整个预训练的loss去求解



## 3. 如何利用BERT等预训练模型完成抽提式文本摘要任务？


+ 引自论文<Fine\-tune BERT for Extractive Summarization>:
    + we insert a [CLS] token before each sentence and a [SEP] token after each sentence. 
    + In vanilla BERT, The [CLS] is used as a symbol to aggregate features from one sentence or a pair of sentences. We modify the model by using multiple [CLS] symbols to get features for sentences ascending the symbol.
    + We use interval segment embeddings to distinguish multiple sentences within a document
    + After obtaining the sentence vectors from BERT, we build several summarization-specific layers stacked on top of the BERT outputs, to capture document-level features for extracting summaries
    + add a linear layer on the BERT outputs and use a sigmoid function to get the predicted score
    

+ 1.对每个句子都加CLS作为输入，输出的为该句子是否作为摘要，是为1，不是为0
如输入X1，X2，X3三个句子，输出Y1，Y2，Y3，其中Y1，Y3为1，则结果为X1X2作为摘要结果
+ 2.对于输入序列长度超过512的，可以通过递归方法去处理，例1000长度的序列，先处理前512个取出最重要的3个句子，再处理剩下的488长度的序列取出最重要的3个句子，最后把这6个拼接输入到训练好的sum模型里，输出最后的结果


## 附加思考题（可做可不做）：

BERT有哪些可以提升的地方呢？

BERT模型在推理上表现不是很好，并不能真正分析出句子之间的逻辑关系，在观点推理理解任务 (ARCT) 的对抗数据集上，BERT成绩就降到了盲猜水平。

1. BERT在预训练过程中使用的数据仅是对单个字符进行屏蔽，但是短语和实体词的屏蔽对中文和英文的也会有作用，ERNIE在预训练时使用的数据是对整个词进行屏蔽，从而学习词与实体的表达。
    + 改进点：Entity-level Masking 预训练和Phrase-level Masking 预训练
    + 例子：ERNIE（Enhanced Representation through kNowledge IntEgration）是百度提出的语义表示模型，同样基于Transformer Encoder，相较于BERT，其预训练过程利用了更丰富的语义知识和更多的语义任务，在多个NLP任务上取得了比BERT等模型更好的效果。(摘自https://aistudio.baidu.com/aistudio/projectdetail/596403 )
    
2. 数据源可以使用多源数据，ERNIE 1.0引入了论坛对话类数据，利用对话语言模式（DLM, Dialogue Language Model）建模Query-Response对话结构，将对话Pair对作为输入，引入Dialogue Embedding标识对话的角色，利用对话响应丢失（DRS, Dialogue Response Loss）学习对话的隐式关系，进一步提升模型的语义表示能力(摘自https://aistudio.baidu.com/aistudio/projectdetail/596403 )

3. BERT只基于MLM和NSP预训练任务，可以尝试增加多语义任务预训练
    + 例子：ERNIE 2.0 的预训练包括了三大类学习任务：
        + 包含词法层任务：学会对句子中的词汇进行预测。
        + 语法层任务：学会将多个句子结构重建，重新排序。
        + 语义层任务：学会判断句子之间的逻辑关系，例如因果关系、转折关系、并列关系等。
        + 结果：ERNIE 2.0模型在英语任务上几乎全面优于BERT和XLNet，在7个GLUE任务上取得了最好的结果；中文任务上，ERNIE 2.0模型在所有9个中文NLP任务上全面优于BERT


